# CPU 2D Gaussian Filter - Weighted Neighbourhood Operations

This demonstrates how kernels apply different weights to neighbouring pixels. The weights follow a 2D Gaussian (bell curve) distribution.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

## 2D Gaussian Function

In [ ]:
def gaussian_2d(x, y, sigma):
    """Formula for 2D Gaussian distribution (isotropic)."""
    coefficient = 1 / (2 * np.pi * sigma**2)
    exponent = -(x**2 + y**2) / (2 * sigma**2)
    return coefficient * np.exp(exponent)

In [ ]:
# visualise 2D Gaussian function
x = np.linspace(-3, 3, 50)
y = np.linspace(-3, 3, 50)
xx, yy = np.meshgrid(x, y)
sigma = 1.0
zz = gaussian_2d(xx, yy, sigma)

fig = plt.figure(figsize=(14, 5))

# 3D surface plot
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot_surface(xx, yy, zz, cmap='viridis', alpha=0.9)
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('G(x,y)')
ax1.set_title(rf'2D Gaussian Surface ($\sigma={sigma}$)')

# 2D heatmap
ax2 = fig.add_subplot(132)
im = ax2.matshow(zz, cmap='viridis', extent=[-3, 3, -3, 3], origin='lower', aspect='equal')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_title('2D Gaussian Heatmap')
ax2.axhline(y=0, color='blue', linewidth=2, linestyle='-', alpha=0.7)
ax2.axvline(x=0, color='red', linewidth=2, linestyle='--', alpha=0.7)
cbar = plt.colorbar(im, ax=ax2, fraction=0.046)
cbar.set_label('Probability Density', rotation=270, labelpad=20)

# cross section
ax3 = fig.add_subplot(133)
center_idx = x.size // 2
ax3.plot(x, zz[center_idx, :], 'b-', linewidth=2, label='Horizontal slice')
ax3.plot(y, zz[:, center_idx], 'r--', linewidth=2, label='Vertical slice')
ax3.set_xlabel('Distance from centre')
ax3.set_ylabel('G(x,0) or G(0,y)')
ax3.set_title('Cross Sections Through Centre')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Compute the 2D Gaussian Weights

In [ ]:
def create_gaussian_kernel_2d(sigma, kernel_size=None):
    """Create 2D Gaussian kernel weights."""
    # input validation
    if sigma <= 0:
        raise ValueError("Sigma must be positive")
    
    # automatically determine kernel size if not provided
    if kernel_size is None:
        # let cover 6-sigma range
        kernel_size = int(6 * sigma + 1)
        # ensure odd size for symmetry
        if kernel_size % 2 == 0:
            kernel_size += 1
    
    # validate kernel size
    if kernel_size < 1:
        raise ValueError("Kernel size must be at least 1.")
    if kernel_size % 2 == 0:
        raise ValueError("Kernel size must be odd for symmetry.")
    
    # create position arrays centred at 0
    half_size = kernel_size // 2
    x = np.arange(-half_size, half_size + 1)
    y = np.arange(-half_size, half_size + 1)
    xx, yy = np.meshgrid(x, y)
    
    # calculate Gaussian weights (without the normalisation coefficient)
    kernel = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    
    # normalise so weights sum to 1
    kernel = kernel / kernel.sum()
    
    return kernel

In [ ]:
def visualise_gaussian_kernel_2d(sigma):
    """Visualise 2D Gaussian kernel weights."""
    kernel = create_gaussian_kernel_2d(sigma)
    kernel_size = kernel.shape[0]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # heatmap
    im = ax1.matshow(kernel, cmap='viridis')
    ax1.set_title(rf'2D Gaussian Kernel ($\sigma={sigma}$, size=${kernel_size}\times{kernel_size}$)')
    ax1.set_xlabel('Column')
    ax1.set_ylabel('Row')
    plt.colorbar(im, ax=ax1, fraction=0.046)
    
    # 3D bar plot
    ax2 = fig.add_subplot(122, projection='3d')
    x_pos, y_pos = np.meshgrid(range(kernel_size), range(kernel_size))
    x_pos = x_pos.flatten()
    y_pos = y_pos.flatten()
    z_pos = np.zeros_like(x_pos)
    dx = dy = 0.8
    dz = kernel.flatten()
    
    ax2.bar3d(x_pos, y_pos, z_pos, dx, dy, dz, color='blue', alpha=0.7)
    ax2.set_xlabel('Column')
    ax2.set_ylabel('Row')
    ax2.set_zlabel('Weight')
    ax2.set_title('Kernel Weights as 3D Bars')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nKernel sum: {kernel.sum():.6f}")
    print(f"Centre weight: {kernel[kernel_size//2, kernel_size//2]:.4f}")
    print(f"Corner weight: {kernel[0, 0]:.6f}")

In [ ]:
for sigma in [0.5, 1.0, 2.0]:
    visualise_gaussian_kernel_2d(sigma)

## Gaussian Filter Implementation

In [ ]:
def gaussian_filter_2d(image, sigma, boundary_mode="zero", quiet=False):
    """Gaussian filter with different padding strategies."""
    # validate input
    if len(image.shape) != 2:
        raise ValueError("Input must be a 2D array")
    
    if boundary_mode not in ("zero", "reflect", "constant"):
        raise ValueError("boundary_mode must be 'zero', 'reflect' or 'constant'")
    
    # create Gaussian kernel
    kernel = create_gaussian_kernel_2d(sigma)
    kernel_size = kernel.shape[0]
    half_size = kernel_size // 2
    
    if not quiet:
        print(f"Gaussian Filter Parameters:")
        print(f"  Sigma: {sigma}")
        print(f"  Kernel size: {kernel_size}x{kernel_size}")
        print(f"  Centre weight: {kernel[half_size, half_size]:.3f}")
        print(f"  Corner weight: {kernel[0, 0]:.3f}")
        print(f"  Sum of weights: {kernel.sum():.6f}")
        print(f"  Boundary mode: {boundary_mode}\n")
        
    # prepare output array (same size as input)
    rows, cols = image.shape
    result = np.zeros((rows, cols), dtype=np.float32)
    
    # each iteration of these outer loops is completely
    # independent - this is why Gaussian filtering is perfect for GPU
    # every thread can process one output pixel without coordination
    for row in range(rows):
        for col in range(cols):
            # accumulate weighted sum
            weighted_sum = 0.0
            weight_sum = 0.0
            
            # show calculations for first pixel
            if not quiet and row == 0 and col == 0:
                print(f"Computing pixel (0,0):")
            
            # apply kernel weights - this is convolution
            for kr in range(kernel_size):
                for kc in range(kernel_size):
                    # calculate position in input image
                    src_row = row + kr - half_size
                    src_col = col + kc - half_size
                    
                    # handle boundaries based on mode
                    if boundary_mode == "zero":
                        # zero padding: use 0 for out-of-bounds
                        if 0 <= src_row < rows and 0 <= src_col < cols:
                            value = image[src_row, src_col]
                        else:
                            value = 0.0
                        weight = kernel[kr, kc]
                    elif boundary_mode == "reflect":
                        # reflect at boundaries
                        ref_row = src_row
                        ref_col = src_col
                        if ref_row < 0:
                            ref_row = -ref_row
                        elif ref_row >= rows:
                            ref_row = 2 * rows - ref_row - 2
                        if ref_col < 0:
                            ref_col = -ref_col
                        elif ref_col >= cols:
                            ref_col = 2 * cols - ref_col - 2
                        value = image[ref_row, ref_col]
                        weight = kernel[kr, kc]
                    else:
                        # constant (use edge value)
                        edge_row = max(0, min(rows - 1, src_row))
                        edge_col = max(0, min(cols - 1, src_col))
                        value = image[edge_row, edge_col]
                        weight = kernel[kr, kc]
                    
                    if weight > 0:
                        contribution = value * weight
                        weighted_sum += contribution
                        weight_sum += weight
                        
                        if not quiet and row == 0 and col == 0 and contribution > 0.001:
                            print(f"  pos ({src_row},{src_col}): "
                                  f"{value:.1f} x {weight:.3f} = {contribution:.2f}")
            
            # normalise by actual weight sum
            if weight_sum > 0:
                result[row, col] = weighted_sum / weight_sum
            else:
                result[row, col] = 0
                
            if not quiet and row == 0 and col == 0:
                print(f"  Result: {weighted_sum:.3f} / {weight_sum:.3f} "
                      f"= {result[row, col]:.1f}\n")
    
    return result

In [ ]:
# create noisy test image
rows, cols = 20, 30
x = np.linspace(0, 4*np.pi, cols)
y = np.linspace(0, 3*np.pi, rows)
xx, yy = np.meshgrid(x, y)

# create pattern
clean_image = (np.sin(xx/2)**2 + np.cos(yy/2)**2) * 100
clean_image = clean_image.astype(np.float32)

# add noise
rng = np.random.default_rng()
noise = rng.normal(0, 15, clean_image.shape)
noisy_image = clean_image + noise

In [ ]:
filtered = gaussian_filter_2d(noisy_image, sigma=1.5)

## Function to Visualise Filtered Image

In [ ]:
def visualise_gaussian_filter_2d(original, filtered, noisy=None):
    """Visualise the effect of Gaussian filtering on 2D images."""
    num_plots = 2 if noisy is None else 3
    fig, ax = plt.subplots(1, num_plots, figsize=(5*num_plots, 5))
    
    # original image
    im0 = ax[0].matshow(original, cmap='viridis')
    ax[0].set_title("Original Image")
    ax[0].set_xlabel("Column")
    ax[0].set_ylabel("Row")
    plt.colorbar(im0, ax=ax[0], fraction=0.046)
    
    if noisy is not None:
        ax_filtered = ax[2]
        # noisy image
        im1 = ax[1].matshow(noisy, cmap='viridis')
        ax[1].set_title("Noisy Image")
        ax[1].set_xlabel("Column")
        ax[1].set_ylabel("Row")
        plt.colorbar(im1, ax=ax[1], fraction=0.046)
    else:
        ax_filtered = ax[1]

    # filtered image
    im2 = ax_filtered.matshow(filtered, cmap='viridis')
    ax_filtered.set_title("After Gaussian Filter")
    ax_filtered.set_xlabel("Column")
    ax_filtered.set_ylabel("Row")
    plt.colorbar(im2, ax=ax_filtered, fraction=0.046)
    
    plt.tight_layout()
    plt.show()

In [ ]:
visualise_gaussian_filter_2d(clean_image, filtered, noisy=noisy_image)

## Handling of Edge
Compare different boundary handling strategies

In [ ]:
# create image with sharp edge to test boundary handling
edge_test = np.zeros((10, 10), dtype=np.float32)
edge_test[:5, :5] = 200  # bright top-left
edge_test[5:, 5:] = 50   # dark bottom-right

fig, ax = plt.subplots(2, 2, figsize=(10, 10))

# original
im0 = ax[0, 0].matshow(edge_test, cmap='gray', vmin=0, vmax=255)
ax[0, 0].set_title("Original")
plt.colorbar(im0, ax=ax[0, 0], fraction=0.046)

# different boundary modes
modes = ["zero", "reflect", "constant"]
positions = [(0, 1), (1, 0), (1, 1)]

for mode, (r, c) in zip(modes, positions):
    filtered = gaussian_filter_2d(edge_test, sigma=1.5, boundary_mode=mode, quiet=True)
    im = ax[r, c].matshow(filtered, cmap='gray', vmin=0, vmax=255)
    ax[r, c].set_title(f"Boundary: {mode}")
    plt.colorbar(im, ax=ax[r, c], fraction=0.046)

for a in ax.flat:
    a.set_xlabel("Column")
    a.set_ylabel("Row")

plt.tight_layout()
plt.show()

## Detailed Convolution Example

In [ ]:
# demonstrate convolution on a small example
small_image = np.array([
    [10,  20,  30,  40],
    [50,  60,  70,  80],
    [90,  100, 110, 120],
    [130, 140, 150, 160]
], dtype=np.float32)

# use simple 3x3 kernel for clarity
simple_kernel = np.array([
    [1, 2, 1],
    [2, 4, 2],
    [1, 2, 1]
], dtype=np.float32) / 16  # normalised

print("Small test image:")
print(small_image)
print("\n3x3 Gaussian kernel (not normalised):")
print(simple_kernel * 16)
print(f"Sum of weights: {simple_kernel.sum():.3f}")

# manually compute one output pixel
print("\nDetailed calculation for pixel (1,1):")
print("\nNeighbourhood:")
neighbourhood = small_image[0:3, 0:3]
print(neighbourhood)

print("\nElement-wise multiplication:")
products = neighbourhood * simple_kernel
for i in range(3):
    for j in range(3):
        print(f"  {int(neighbourhood[i,j]):3d} × {simple_kernel[i,j]:.3f} = {products[i,j]:6.2f}")

print(f"\nSum of products: {products.sum():.2f}")
print(f"This is the output value at position (1,1)")

## Comparing Different Sigma

In [ ]:
# create test image with fine details
detail_image = np.zeros((30, 30), dtype=np.float32)
# add various features
detail_image[5:10, 5:10] = 200    # small square
detail_image[15:25, 15:25] = 150  # large square
detail_image[10:12, :] = 100       # horizontal line
detail_image[:, 20:22] = 100       # vertical line

# add some noise
noise = rng.normal(0, 10, detail_image.shape)
detail_noisy = np.clip(detail_image + noise, 0, 255)

# apply different sigmas
sigmas = (0.5, 1.0, 2.0, 3.0)
fig, ax = plt.subplots(2, 3, figsize=(15, 10))

# original and noisy
im00 = ax[0, 0].matshow(detail_image, cmap='gray', vmin=0, vmax=255)
ax[0, 0].set_title("Original")
plt.colorbar(im00, ax=ax[0, 0], fraction=0.046)

im01 = ax[0, 1].matshow(detail_noisy, cmap='gray', vmin=0, vmax=255)
ax[0, 1].set_title("With Noise")
plt.colorbar(im01, ax=ax[0, 1], fraction=0.046)

# different sigmas
positions = [(0, 2), (1, 0), (1, 1), (1, 2)]
for sigma, (r, c) in zip(sigmas, positions):
    filtered = gaussian_filter_2d(detail_noisy, sigma=sigma, quiet=True)
    im = ax[r, c].matshow(filtered, cmap='gray', vmin=0, vmax=255)
    ax[r, c].set_title(rf"$\sigma={sigma}$")
    plt.colorbar(im, ax=ax[r, c], fraction=0.046)

for a in ax.flat:
    a.set_xlabel("Column")
    a.set_ylabel("Row")

plt.tight_layout()
plt.show()

print("Observations about sigmas:")
print("- Small (0.5): Minimal smoothing, preserves fine details")
print("- Medium (1-2): Good balance of noise reduction and detail preservation")
print("- Large (3+): Strong smoothing, loses fine details like thin lines")